In [1]:
import numpy as np
import pandas as pd
import os
print(os.listdir("../input"))
from re import sub
from decimal import Decimal
from sklearn.preprocessing import MultiLabelBinarizer

['calendar.csv', 'listings.csv', 'reviews.csv']


# Grabbing Airbnb Listings Data

We chose specific data points from the listings data that we thought were easy for us to categorize during our data cleaning process, and, intuitively, seemed most relevant to predicting the pricing.

Data points:
- host_is_superhost
- neighbourhood_group_cleansed
- property_type
- room_type
- latitude
- longitude
- guests_included
- bathrooms
- bedrooms
- beds
- bed_type
- amenities
- price
- cleaning_fee
- instant_bookable
- cancellation_policy

In [2]:
listings = pd.read_csv('../input/listings.csv')
ld = listings.loc[:,['host_is_superhost','neighbourhood_group_cleansed', 'property_type', 'room_type', 'latitude', 'longitude', 'guests_included', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price', 'cleaning_fee', 'instant_bookable', 'cancellation_policy']] 

In [3]:
display(listings.head())

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,...,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,...,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,...,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,Our house is located just 5 short blocks to To...,A bus stop is just 2 blocks away. Easy bus a...,NaN,NaN,

In [4]:
ld.count()

host_is_superhost               3816
neighbourhood_group_cleansed    3818
property_type                   3817
room_type                       3818
latitude                        3818
longitude                       3818
guests_included                 3818
bathrooms                       3802
bedrooms                        3812
beds                            3817
bed_type                        3818
amenities                       3818
price                           3818
cleaning_fee                    2788
instant_bookable                3818
cancellation_policy             3818
dtype: int64

# Removing all the listings with missing values

In this step, we are looking for all the listings containing missing values.  We will remove them, and store them in another dataframe. 

In [5]:
ld = ld.dropna(subset=['host_is_superhost','neighbourhood_group_cleansed', 'property_type', 'room_type', 'latitude', 'longitude', 'guests_included', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price', 'instant_bookable', 'cancellation_policy']) 
ld.count()

host_is_superhost               3793
neighbourhood_group_cleansed    3793
property_type                   3793
room_type                       3793
latitude                        3793
longitude                       3793
guests_included                 3793
bathrooms                       3793
bedrooms                        3793
beds                            3793
bed_type                        3793
amenities                       3793
price                           3793
cleaning_fee                    2771
instant_bookable                3793
cancellation_policy             3793
dtype: int64

In [6]:
ld['cleaning_fee'] = ld['cleaning_fee'].fillna(0)

In [7]:
ld['cleaning_fee'].head()

0          0
1     $40.00
2    $300.00
3          0
4    $125.00
Name: cleaning_fee, dtype: object

In [8]:
ld.head()

,host_is_superhost,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,guests_included,bathrooms,bedrooms,beds,bed_type,amenities,price,cleaning_fee,instant_bookable,cancellation_policy
0,f,Queen Anne,Apartment,Entire home/apt,47.636289,-122.371025,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,f,moderate
1,t,Queen Anne,Apartment,Entire home/apt,47.639123,-122.365666,1,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,f,strict
2,f,Queen Anne,House,Entire home/apt,47.629724,-122.369483,10,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,f,strict
3,f,Queen Anne,Apartment,Entire home/apt,47.638473,-122.369279,1,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,f,flexible
4,f,Queen Anne,House,Entire home/apt,47.632918,-122.372471,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,f,strict


## Column 1: host_is_superhost
- Boolean declaring whether host fulfills Airbnb's superhost requirements: https://www.airbnb.ca/help/article/829/how-do-i-become-a-superhost
- Convert `True = 1` and `False = 0`

In [9]:
ld.loc[ld.loc[:, 'host_is_superhost'] == 't', 'host_is_superhost'] = 1
ld.loc[ld.loc[:, 'host_is_superhost'] == 'f', 'host_is_superhost'] = 0

In [10]:
ld.head()

,host_is_superhost,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,guests_included,bathrooms,bedrooms,beds,bed_type,amenities,price,cleaning_fee,instant_bookable,cancellation_policy
0,0,Queen Anne,Apartment,Entire home/apt,47.636289,-122.371025,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,f,moderate
1,1,Queen Anne,Apartment,Entire home/apt,47.639123,-122.365666,1,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,f,strict
2,0,Queen Anne,House,Entire home/apt,47.629724,-122.369483,10,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,f,strict
3,0,Queen Anne,Apartment,Entire home/apt,47.638473,-122.369279,1,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,f,flexible
4,0,Queen Anne,House,Entire home/apt,47.632918,-122.372471,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,f,strict


## Column 2: neighbourhood_group_cleansed
- neighborhood category string that states which neighbourhood it belongs to

In [11]:
# all the possible values
set(ld['neighbourhood_group_cleansed'])

{'Ballard',
 'Beacon Hill',
 'Capitol Hill',
 'Cascade',
 'Central Area',
 'Delridge',
 'Downtown',
 'Interbay',
 'Lake City',
 'Magnolia',
 'Northgate',
 'Other neighborhoods',
 'Queen Anne',
 'Rainier Valley',
 'Seward Park',
 'University District',
 'West Seattle'}

In [12]:
neighbourhood = pd.get_dummies(ld['neighbourhood_group_cleansed'].str.lower().str.replace(' ', '_'))

In [13]:
neighbourhood.head()

,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [14]:
ld_1 = pd.merge(ld, neighbourhood, left_index=True, right_index=True)
ld_1 = ld_1.drop('neighbourhood_group_cleansed', 1)

In [15]:
ld_1.head()

,host_is_superhost,property_type,room_type,latitude,longitude,guests_included,bathrooms,bedrooms,beds,bed_type,amenities,price,cleaning_fee,instant_bookable,cancellation_policy,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle
0,0,Apartment,Entire home/apt,47.636289,-122.371025,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,f,moderate,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,Apartment,Entire home/apt,47.639123,-122.365666,1,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,f,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,House,Entire home/apt,47.629724,-122.369483,10,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,f,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,Apartment,Entire home/apt,47.638473,-122.369279,1,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,f,flexible,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,House,Entire home/apt,47.632918,-122.372471,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,f,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


 ## Column 3: Property type
 
 - Column indicates which property type it is(ex. house, apartment, etc)

In [16]:
property_type = pd.get_dummies(ld_1['property_type'])

In [17]:
property_type.head()

,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [18]:
ld_2 = pd.merge(ld_1, property_type, left_index=True, right_index=True)
ld_2 = ld_2.drop('property_type', 1)

In [19]:
ld_2.head()

,host_is_superhost,room_type,latitude,longitude,guests_included,bathrooms,bedrooms,beds,bed_type,amenities,price,cleaning_fee,instant_bookable,cancellation_policy,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt
0,0,Entire home/apt,47.636289,-122.371025,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,f,moderate,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Entire home/apt,47.639123,-122.365666,1,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,f,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,Entire home/apt,47.629724,-122.369483,10,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,f,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,Entire home/apt,47.638473,-122.369279,1,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,f,flexible,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,Entire home/apt,47.632918,-122.372471,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,f,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## Column 4: Room type
 
 - Column indicates which room type it is(ex. Entire home/apt )

In [20]:
room_type = pd.get_dummies(ld_2['room_type'])

In [21]:
room_type.head()

,Entire home/apt,Private room,Shared room
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [22]:
ld_3= pd.merge(ld_2, room_type, left_index=True, right_index=True)
ld_3 = ld_3.drop('room_type', 1)

## Column 5: Bed type
 
 - Column states what kind of bed the listing has(ex. Real Bed, Futon, etc )

In [23]:
bed_type = pd.get_dummies(ld_3['bed_type'])

In [24]:
bed_type.head()

,Airbed,Couch,Futon,Pull-out Sofa,Real Bed
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [25]:
ld_4= pd.merge(ld_3, bed_type, left_index=True, right_index=True)
ld_4 = ld_4.drop('bed_type', 1)

## Column 6: Instant Bookable
 
- Boolean declaring whether or not the listing can be instant booked. 
- Convert `True = 1` and `False = 0`

In [26]:
ld_4.loc[ld_4.loc[:, 'instant_bookable'] == 't', 'instant_bookable'] = 1
ld_4.loc[ld_4.loc[:, 'instant_bookable'] == 'f', 'instant_bookable'] = 0

In [27]:
ld_4.head()

,host_is_superhost,latitude,longitude,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,cancellation_policy,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed
0,0,47.636289,-122.371025,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,0,moderate,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,1,47.639123,-122.365666,1,1.0,1.0,1.0,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,0,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,47.629724,-122.369483,10,4.5,5.0,7.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,0,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0,47.638473,-122.369279,1,1.0,0.0,2.0,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,0,flexible,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,47.632918,-122.372471,6,2.0,3.0,3.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,0,strict,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1


## Column 7: Cancellation policy
- Column indicates which kind of standardlized cancellation policy the host chooses.
- There are three cancellation policies - flexible, moderate and strict 

In [28]:
cancellation = pd.get_dummies(ld_4['cancellation_policy'])

In [29]:
cancellation.head()

,flexible,moderate,strict
0,0,1,0
1,0,0,1
2,0,0,1
3,1,0,0
4,0,0,1


In [30]:
ld_5= pd.merge(ld_4, cancellation, left_index=True, right_index=True)
ld_5 = ld_5.drop('cancellation_policy', 1)

# Column 8: Guest included 
- column states the number of guests can be accomodated for each listing
- we need to normalize the value to 0-1

In [31]:
ld_5['guests_included'].max()

15

In [32]:
def normalizing(column):
    new_column = (column - column.min()) / (column.max() - column.min())
    return new_column

In [33]:
ld_5['guests_included'] = normalizing(ld_5['guests_included'])

In [34]:
ld_5.head()

,host_is_superhost,latitude,longitude,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict
0,0,47.636289,-122.371025,0.133333,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
1,1,47.639123,-122.365666,0.066667,1.0,1.0,1.0,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
2,0,47.629724,-122.369483,0.666667,4.5,5.0,7.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
3,0,47.638473,-122.369279,0.066667,1.0,0.0,2.0,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,0,47.632918,-122.372471,0.400000,2.0,3.0,3.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1


# Column 8, 9, 10: bathrooms, bedrooms, beds
- column states the number of bathrooms, bedrooms, and beds in each listing
- normalize the value to 0-1

In [35]:
ld_5['bathrooms'] = normalizing(ld_5['bathrooms'])

In [36]:
ld_5['bedrooms'] = normalizing(ld_5['bedrooms'])

In [37]:
ld_5['beds'] = normalizing(ld_5['beds'])

In [38]:
ld_5.head()

,host_is_superhost,latitude,longitude,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict
0,0,47.636289,-122.371025,0.133333,0.1250,0.142857,0.000000,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
1,1,47.639123,-122.365666,0.066667,0.1250,0.142857,0.000000,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
2,0,47.629724,-122.369483,0.666667,0.5625,0.714286,0.428571,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
3,0,47.638473,-122.369279,0.066667,0.1250,0.000000,0.071429,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,0,47.632918,-122.372471,0.400000,0.2500,0.428571,0.142857,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1


# Column 11, 12: Longitude and Latitude 
- column states the longitude and latitude of each listing 
- we can use these two values, and map them to x, y and z coordinates. In this way we can make sure close points in the 3D space are close to each other. 
- x = cos(lat) * cos(lon)
- y = cos(lat) * sin(lon), 
- z = sin(lat) 

In [39]:
## seattle airport: 47.4502° N, 122.3088° W
airport_lat = 47.4502
airport_lon = -122.3088

## downtown: 47.6050° N, 122.3344° W
dt_lat = 47.6050
dt_lon = -122.3344

## pike place: 47.6101° N, 122.3421° W
pp_lat = 47.6101
pp_lon = -122.3421

## seattle amazon headquarter: 47.6062° N, 122.3321° W
amazon_lat = 47.6062
amazon_lon = -122.3321

## longitude and latitude in datasets
lat_data = ld_5['latitude']
lon_data = ld_5['longitude']

In [40]:
lat_data[1]

47.63912312136253

In [41]:
lon_data[1]

-122.36566646439582

In [42]:
airport_lat

47.4502

In [43]:
AVG_EARTH_RADIUS = 6371

In [44]:
def haversine_array(lat1, lng1, ld_5):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, ld_5['latitude'], ld_5['longitude']))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arctan2(np.sqrt(d), np.sqrt(1-d))
    return h

In [45]:
ld_5['d_airport'] = haversine_array(airport_lat, airport_lon, ld_5)
ld_5['d_downtown'] = haversine_array(dt_lat, dt_lon, ld_5)
ld_5['d_pikeplace'] = haversine_array(pp_lat, pp_lon, ld_5)
ld_5['d_amazon'] = haversine_array(amazon_lat, amazon_lon, ld_5)


In [46]:
ld_5.head()

,host_is_superhost,latitude,longitude,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,d_airport,d_downtown,d_pikeplace,d_amazon
0,0,47.636289,-122.371025,0.133333,0.1250,0.142857,0.000000,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,21.212736,4.431697,3.630386,4.439052
1,1,47.639123,-122.365666,0.066667,0.1250,0.142857,0.000000,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,21.436526,4.459606,3.678898,4.441933
2,0,47.629724,-122.369483,0.666667,0.5625,0.714286,0.428571,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,20.475301,3.804344,2.995638,3.833194
3,0,47.638473,-122.369279,0.066667,0.1250,0.000000,0.071429,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,21.421534,4.548324,3.755367,4.543407
4,0,47.632918,-122.372471,0.400000,0.2500,0.428571,0.142857,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,20.871910,4.216570,3.408671,4.240533


In [47]:
ld_5['d_airport'] = normalizing(ld_5['d_airport'])
ld_5['d_downtown'] = normalizing(ld_5['d_downtown'])
ld_5['d_pikeplace'] = normalizing(ld_5['d_pikeplace'])
ld_5['d_amazon'] = normalizing(ld_5['d_amazon'])


In [48]:
ld_5.head()

,host_is_superhost,latitude,longitude,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,d_airport,d_downtown,d_pikeplace,d_amazon
0,0,47.636289,-122.371025,0.133333,0.1250,0.142857,0.000000,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.567750,0.301354,0.260323,0.306945
1,1,47.639123,-122.365666,0.066667,0.1250,0.142857,0.000000,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.576957,0.303310,0.263846,0.307148
2,0,47.629724,-122.369483,0.666667,0.5625,0.714286,0.428571,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.537412,0.257394,0.214225,0.264340
3,0,47.638473,-122.369279,0.066667,0.1250,0.000000,0.071429,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.576340,0.309527,0.269400,0.314284
4,0,47.632918,-122.372471,0.400000,0.2500,0.428571,0.142857,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.553728,0.286280,0.244221,0.292985


In [49]:
ld_6 = ld_5.drop('latitude', 1)
ld_6.head()

,host_is_superhost,longitude,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,d_airport,d_downtown,d_pikeplace,d_amazon
0,0,-122.371025,0.133333,0.1250,0.142857,0.000000,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.567750,0.301354,0.260323,0.306945
1,1,-122.365666,0.066667,0.1250,0.142857,0.000000,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.576957,0.303310,0.263846,0.307148
2,0,-122.369483,0.666667,0.5625,0.714286,0.428571,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.537412,0.257394,0.214225,0.264340
3,0,-122.369279,0.066667,0.1250,0.000000,0.071429,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.576340,0.309527,0.269400,0.314284
4,0,-122.372471,0.400000,0.2500,0.428571,0.142857,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.553728,0.286280,0.244221,0.292985


In [50]:
ld_7 = ld_6.drop('longitude', 1)

In [51]:
ld_7.head()

,host_is_superhost,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,d_airport,d_downtown,d_pikeplace,d_amazon
0,0,0.133333,0.1250,0.142857,0.000000,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$85.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.567750,0.301354,0.260323,0.306945
1,1,0.066667,0.1250,0.142857,0.000000,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$150.00,$40.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.576957,0.303310,0.263846,0.307148
2,0,0.666667,0.5625,0.714286,0.428571,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$975.00,$300.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.537412,0.257394,0.214225,0.264340
3,0,0.066667,0.1250,0.000000,0.071429,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",$100.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.576340,0.309527,0.269400,0.314284
4,0,0.400000,0.2500,0.428571,0.142857,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$450.00,$125.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.553728,0.286280,0.244221,0.292985


# Column 13: price, cleaning fee
- column includes the avrage price and cleaning_fee per night for each listing

In [52]:
ld_7['price'] = ld_7['price'].replace('[\$,]','',regex=True).astype(float)
ld_7['cleaning_fee'] = ld_7['cleaning_fee'].replace('[\$,]','',regex=True).astype(float)
ld_7['price'] = normalizing(ld_7['price'])
ld_7['cleaning_fee'] = normalizing(ld_7['cleaning_fee'] )

In [53]:
ld_7.head()

,host_is_superhost,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,d_airport,d_downtown,d_pikeplace,d_amazon
0,0,0.133333,0.1250,0.142857,0.000000,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0.066327,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.567750,0.301354,0.260323,0.306945
1,1,0.066667,0.1250,0.142857,0.000000,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.132653,0.133333,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.576957,0.303310,0.263846,0.307148
2,0,0.666667,0.5625,0.714286,0.428571,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0.974490,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.537412,0.257394,0.214225,0.264340
3,0,0.066667,0.1250,0.000000,0.071429,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",0.081633,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.576340,0.309527,0.269400,0.314284
4,0,0.400000,0.2500,0.428571,0.142857,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",0.438776,0.416667,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.553728,0.286280,0.244221,0.292985


In [54]:
ld_8 = ld_7.copy()
ld_8.head()

,host_is_superhost,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,d_airport,d_downtown,d_pikeplace,d_amazon
0,0,0.133333,0.1250,0.142857,0.000000,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0.066327,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.567750,0.301354,0.260323,0.306945
1,1,0.066667,0.1250,0.142857,0.000000,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.132653,0.133333,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.576957,0.303310,0.263846,0.307148
2,0,0.666667,0.5625,0.714286,0.428571,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",0.974490,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.537412,0.257394,0.214225,0.264340
3,0,0.066667,0.1250,0.000000,0.071429,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",0.081633,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.576340,0.309527,0.269400,0.314284
4,0,0.400000,0.2500,0.428571,0.142857,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",0.438776,0.416667,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.553728,0.286280,0.244221,0.292985


# Column 14: Amenities 
- column indudes all the amentities for each listing

In [55]:
ld_8["amenities"] = ld_8["amenities"].str.lower().str.replace('{','').str.replace('}','').str.replace('"','').str.replace(' ','_').str.split(',')
ld_8.head()



,host_is_superhost,guests_included,bathrooms,bedrooms,beds,amenities,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,Yurt,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,d_airport,d_downtown,d_pikeplace,d_amazon
0,0,0.133333,0.1250,0.142857,0.000000,"[tv, cable_tv, internet, wireless_internet, ai...",0.066327,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.567750,0.301354,0.260323,0.306945
1,1,0.066667,0.1250,0.142857,0.000000,"[tv, internet, wireless_internet, kitchen, fre...",0.132653,0.133333,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.576957,0.303310,0.263846,0.307148
2,0,0.666667,0.5625,0.714286,0.428571,"[tv, cable_tv, internet, wireless_internet, ai...",0.974490,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.537412,0.257394,0.214225,0.264340
3,0,0.066667,0.1250,0.000000,0.071429,"[internet, wireless_internet, kitchen, indoor_...",0.081633,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.576340,0.309527,0.269400,0.314284
4,0,0.400000,0.2500,0.428571,0.142857,"[tv, cable_tv, internet, wireless_internet, ki...",0.438776,0.416667,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0.553728,0.286280,0.244221,0.292985


In [56]:
mlb = MultiLabelBinarizer()
final_df = ld_8.join(pd.DataFrame(mlb.fit_transform(ld_8.pop('amenities')),
                          columns=mlb.classes_,
                          index=ld_8.index))
final_df.head()

,host_is_superhost,guests_included,bathrooms,bedrooms,beds,price,cleaning_fee,instant_bookable,ballard,beacon_hill,capitol_hill,cascade,central_area,delridge,downtown,interbay,lake_city,magnolia,northgate,other_neighborhoods,queen_anne,rainier_valley,seward_park,university_district,west_seattle,Apartment,Bed & Breakfast,Boat,Bungalow,Cabin,Camper/RV,Chalet,Condominium,Dorm,House,Loft,Other,Tent,Townhouse,Treehouse,...,air_conditioning,breakfast,buzzer/wireless_intercom,cable_tv,carbon_monoxide_detector,cat(s),dog(s),doorman,dryer,elevator_in_building,essentials,family/kid_friendly,fire_extinguisher,first_aid_kit,free_parking_on_premises,gym,hair_dryer,hangers,heating,hot_tub,indoor_fireplace,internet,iron,kitchen,laptop_friendly_workspace,lock_on_bedroom_door,other_pet(s),pets_allowed,pets_live_on_this_property,pool,safety_card,shampoo,smoke_detector,smoking_allowed,suitable_for_events,tv,washer,washer_/_dryer,wheelchair_accessible,wireless_internet
0,0,0.133333,0.1250,0.142857,0.000000,0.066327,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
1,1,0.066667,0.1250,0.142857,0.000000,0.132653,0.133333,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0,1,1,1,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,1
2,0,0.666667,0.5625,0.714286,0.428571,0.974490,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,1,1,1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,1,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
3,0,0.066667,0.1250,0.000000,0.071429,0.081633,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1
4,0,0.400000,0.2500,0.428571,0.142857,0.438776,0.416667,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1


In [57]:
# Exporting as csv for later use
final_df.to_csv('aps360_airbnb2.csv')